# tensorflow Data
________________________________________________________________________________________________________________________
## 머신러닝의 시간 중 7~80%는 데이터분석, 전처리, 파이프라인을 만드는 과정에서 소모됨
## 이를 위해 텐플로는 Dataset API를 활용하여 단순한 연구뿐만 아니라 서비스화를 위한 모듈 제공
## 다른 모듈들과 달리 tf.data모듈은 셔플, 반복, 배치 설정등과 같은 기능들이 구현되있고 
## 최적화가 되어있음

### tf.data

In [7]:
# 라이브러리 불러오기 및 1.0코드를 2.0에서 사용할 수 있도록 설정
import os
import tensorflow.compat.v1 as tf
import numpy as np

from tensorflow.keras import preprocessing

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
# 텍스트 데이터, 각 데이터의 라벨 정의
samples = ['너 오늘 이뻐 보인다', 
           '나는 오늘 기분이 더러워', 
           '끝내주는데, 좋은 일이 있나봐', 
           '나 좋은 일이 생겼어', 
           '아 오늘 진짜 짜증나', 
           '환상적인데, 정말 좋은거 같아']

label = [[1], [0], [1], [1], [0], [1]]

In [9]:
MAX_LEN = 4

In [10]:
# 데이터 전처리 실행(후에 자세히 다룰 예정)
# 간단히 설명하자면 텍스트로 구성된 데이터에서 각 단어를 단어의 인덱스로 바꿈(텍스트 데이터 수치화)
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

sequences = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_LEN, padding='post')

word_index = tokenizer.word_index

In [11]:
#결과 확인
print("수치화된 텍스트 데이터: \n",sequences)
print("각 단어의 인덱스: \n", word_index) 
print("라벨: ", label) 
# 결과를 보면 가장 많이 쓰인단어를 우선으로 숫자가 매겨짐을 확인할 수 있다.

수치화된 텍스트 데이터: 
 [[ 4  1  5  6]
 [ 7  1  8  9]
 [10  2  3 11]
 [12  2  3 13]
 [14  1 15 16]
 [17 18 19 20]]
각 단어의 인덱스: 
 {'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}
라벨:  [[1], [0], [1], [1], [0], [1]]


In [12]:
# tf.data 활용

# 주어진 데이터 sequences와 label을 묶어서 조각으로 만들고 함께 사용할 수 있게 해주는 함수
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
# 정해진 데이터 셋을 이터레이터 형식으로 정의
iterator = dataset.make_one_shot_iterator()
# 데이터가 하나씩 나오도록 get_next()사용
next_data = iterator.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [13]:
# 텐서플로의 세션을 실행한 후 반복문을 사용해 next_data를 계속해서 불러온다
# 더 이상 불러올 데이터가 없는 경우 break
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))


In [14]:
# tf.data의 배치(batch) 기능

# 배치 크기 정의
BATCH_SIZE = 2

# 배치 크기 설정 후 데이터 불러오기
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.batch(BATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [15]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break
            
# 한번에 2개의 데이터가 나오는 것을 확인 할 수 있음

(array([[4, 1, 5, 6],
       [7, 1, 8, 9]]), array([[1],
       [0]]))
(array([[10,  2,  3, 11],
       [12,  2,  3, 13]]), array([[1],
       [1]]))
(array([[14,  1, 15, 16],
       [17, 18, 19, 20]]), array([[0],
       [1]]))


In [16]:
# tf.data의 셔플(shuffle) 기능
# 셔플 함수를 사용하면 데이터를 임의의 순서대로 불러옴
# 인자값은 데이터의 전체 길이
# 이후 모델 성능에 크게 기여할 수 있음
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.shuffle(len(sequences))
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [17]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break
# 결과를 보면 값이 임의의 순서로 출력됨을 확인할 수 있음

(array([10,  2,  3, 11]), array([1]))
(array([4, 1, 5, 6]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([7, 1, 8, 9]), array([0]))
(array([17, 18, 19, 20]), array([1]))


In [18]:
# tf.data의 반복(repeat) 기능
# 모델을 데이터를 사용해 학습시킬 때 데이터를 한 번씩만 사용하는 것이 아니라 여러 번 사용해서 학습함
# 이때, 전체 데이터를 몇 번 사용하는지를 지칭하는 말이 Epoch
# 기존의 방법은 데이터가 한번씩만 사용되고 더이상 불러오지 않지만, 이를 설정하면 정의한 만큼 데이터를 계속 불러올 수 있음
# 이를 위해 사용하는 함수가 repeat함수이다

EPOCH = 2

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.repeat(EPOCH)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [19]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break
            
# 결과를 보면 Epoch만큼 데이터가 반복하여 출력된다.

(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))
(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))


In [20]:
# tf.data의 매핑(mapping) 기능
# 모델에 따라 입력값이 두개 이상이 될 수 있는데, 이때 라벨을 제외한 나머지 데이터를 하나의 입력값으로 묶기 위한 과정을 매핑이라 한다.
# tf.data의 매핑 기능을 사용하기 위해선 추가적인 매핑 함수를 정의한 후, dataset에 정의한 함수를 적용하면 된다.

# 매핑 함수 정의
def mapping_fn(X, Y=None):
    input = {'x': X}
    label = Y
    return input, label

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [22]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break
# 결과를 보면 입력 데이터가 'x'라는 키 값을 가지고있는 딕셔너리 구조이다.

({'x': array([4, 1, 5, 6])}, array([1]))
({'x': array([7, 1, 8, 9])}, array([0]))
({'x': array([10,  2,  3, 11])}, array([1]))
({'x': array([12,  2,  3, 13])}, array([1]))
({'x': array([14,  1, 15, 16])}, array([0]))
({'x': array([17, 18, 19, 20])}, array([1]))


In [23]:
# tf.data에서 위의 기능들 중 매핑, 셔플, 배치 적용한 경우

#매핑
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
#셔플
dataset = dataset.shuffle(len(sequences))
#배치
dataset = dataset.batch(BATCH_SIZE) 

iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [24]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break
# 결과를 보면 'x'를 키값으로 가지는 2개의 데이터가 한번에 임의의 순서로 딕셔너리 구조로 출력되는것을 확인할 수 있다.

({'x': array([[10,  2,  3, 11],
       [12,  2,  3, 13]])}, array([[1],
       [1]]))
({'x': array([[17, 18, 19, 20],
       [14,  1, 15, 16]])}, array([[1],
       [0]]))
({'x': array([[7, 1, 8, 9],
       [4, 1, 5, 6]])}, array([[0],
       [1]]))


In [3]:
# tf.data에서 위의 기능들 중 매핑, 셔플, 배치, 반복 모두 적용한 경우

BATCH_SIZE = 2
EPOCH = 2

def mapping_fn(X, Y=None):
    input = {'x': X}
    label = Y
    return input, label

# 매핑
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
# 셔플
dataset = dataset.shuffle(len(sequences))
# 배치
dataset = dataset.batch(BATCH_SIZE) 
# 반복
dataset = dataset.repeat(EPOCH)

iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

NameError: name 'tf' is not defined